# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('InsertTableName', engine)
X = df['message']
y = df.iloc[:,4:]

In [3]:
categories_name = y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    
    result = re.sub(r'\d+', '', text)
    tokens = word_tokenize(result)
    rm_pun= [word for word in tokens if word.isalnum()]
    tokens = [w for w in rm_pun if not w in stop_words]
    clean_tokens = [porter.stem(t) for t in tokens]
    #clean_text = ' '.join(clean_tokens)
    
    return clean_tokens


In [5]:
#X_cleaned = X.apply(tokenize)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split

import numpy as np

In [7]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
               #,
               #'bootstrap': bootstrap}

In [8]:
pipeline =  Pipeline([
     ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
     ('clf', MultiOutputClassifier(RandomForestClassifier())),
 ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score

In [13]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))

Accuracy:  0.15698924731182795
Precision:  0.4153669060278082
Recall:  0.2886698545915878


/Users/bat/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bat/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]),)

related
              precision    recall  f1-score   support

         0.0       0.38      0.07      0.12      1615
         1.0       0.76      0.96      0.85      4895

    accuracy                           0.74      6510
   macro avg       0.57      0.52      0.48      6510
weighted avg       0.67      0.74      0.67      6510

request
              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90      5372
         1.0       0.45      0.06      0.11      1138

    accuracy                           0.82      6510
   macro avg       0.64      0.52      0.50      6510
weighted avg       0.77      0.82      0.76      6510

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6478
         1.0       0.00      0.00      0.00        32

    accuracy                           1.00      6510
   macro avg       0.50      0.50      0.50      6510
weighted avg       0.99      1.00      0.99      651

/Users/bat/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6183
         1.0       0.12      0.01      0.01       327

    accuracy                           0.95      6510
   macro avg       0.53      0.50      0.49      6510
weighted avg       0.91      0.95      0.92      6510

search_and_rescue
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6310
         1.0       0.00      0.00      0.00       200

    accuracy                           0.97      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.94      0.97      0.95      6510

security
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6385
         1.0       0.00      0.00      0.00       125

    accuracy                           0.98      6510
   macro avg       0.49      0.50      0.50      6510
weighted avg       0.96      0.98      0.97    

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6162
         1.0       0.14      0.00      0.01       348

    accuracy                           0.95      6510
   macro avg       0.54      0.50      0.49      6510
weighted avg       0.90      0.95      0.92      6510

direct_report
              precision    recall  f1-score   support

         0.0       0.81      0.99      0.89      5226
         1.0       0.46      0.04      0.08      1284

    accuracy                           0.80      6510
   macro avg       0.63      0.52      0.48      6510
weighted avg       0.74      0.80      0.73      6510



### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
parameters = {
        "clf__estimator__n_estimators": [15, 20, 35, 50, 100],
        "clf__estimator__max_depth": [None, 50, 100],
        "clf__estimator__max_features": ["sqrt", "auto"]
    }
cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

In [17]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'tfidf', 'clf', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_scor

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))

In [ ]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
pipeline =  Pipeline([
     ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
     ('clf', MultiOutputClassifier(AdaBoostClassifier())),
 ])

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))

In [ ]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

### 9. Export your model as a pickle file

In [ ]:
import joblib
filename = 'saved_model.pkl'
joblib.dump(cv.best_estimator_, filename)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.